In [1]:
#!/usr/bin/env python

from selenium import webdriver
import pandas as pd
import re
from bs4 import BeautifulSoup
from collections import defaultdict
import time
import random

In [2]:
ls

Data_Scraping_Prototype.ipynb           cast_info_list_start_to_1500.pickle
Data_Scraping_Prototype_Selenium.ipynb  clean_movie_df.pickle
Feature Engineering.ipynb               cleaning_prototype.ipynb
README.md                               geckodriver.log
Regression prototype-V2.ipynb           movie_info.pickle
Regression prototype.ipynb              movie_stubs.pickle
Untitled.ipynb                          processed_movie_df.pickle
cast_info.pickle                        usholidays.csv
cast_info_list1500_to_end.pickle


In [3]:
import pickle 
movie_df = pickle.load(open('movie_stubs.pickle', 'rb'))

In [4]:
movie_df.link_stub

Avengers: Endgame                            /release/rl3059975681/?ref_=bo_yld_table_1
The Lion King                                /release/rl3321923073/?ref_=bo_yld_table_2
Toy Story 4                                  /release/rl3798500865/?ref_=bo_yld_table_3
Frozen II                                    /release/rl2424210945/?ref_=bo_yld_table_4
Captain Marvel                               /release/rl3009644033/?ref_=bo_yld_table_5
                                                               ...                     
Tarzan                                     /release/rl2742388225/?ref_=bo_yld_table_337
Music of the Heart                          /release/rl761562625/?ref_=bo_yld_table_344
The Messenger: The Story of Joan of Arc    /release/rl1986168321/?ref_=bo_yld_table_350
Snatch                                      /release/rl141592065/?ref_=bo_yld_table_401
The Gift                                    /release/rl945063425/?ref_=bo_yld_table_411
Name: link_stub, Length: 4279, d

In [5]:
def get_cast_info(link):
    
    base_url = "https://www.boxofficemojo.com"
    url = base_url+link
    #linux browser = webdriver.Firefox()
    browser = webdriver.Chrome(executable_path='/Applications/chromedriver')
    browser.get(url)
    time.sleep(0.1*random.random())
    browser.find_element_by_link_text('Title Summary').click()
    browser.find_element_by_link_text('Cast and Crew').click()

    soup = BeautifulSoup(browser.page_source)
               
    crew_dict = defaultdict(list)
    
    #find Table Element 
    tables = soup.find_all('table')

    crew_rows = tables[0].find_all('tr')
    for row in crew_rows[1:]:
        curr_row = row.find_all('td')
        role = curr_row[1].text.rstrip()
        name = curr_row[0].text.rstrip()
        #add to crew dictionairy
        crew_dict[role].append(name)


    actor_rows = tables[1].find_all('tr')
    for row in actor_rows[1:]:
        curr_row = row.find_all('td')
        name = curr_row[0].text.rstrip()
        crew_dict['Actors'].append(name)

    cast_dict = {}
    title = soup.find('h1').text
    cast_dict['title'] = title
    
    for key in crew_dict:
        cast_dict[key] = ', '.join(crew_dict[key])
        
    browser.quit()
    return cast_dict
    

In [29]:
cast_info_list = []

In [30]:
c = 0
for link in movie_df.link_stub:
    try:
        cast_info_list.append(get_cast_info(link))
        browser.quit()
    except:
        c+=1
        print('{} rows missed', end="\r", flush=True)
    print('{} rows completed'.format(len(cast_info_list)), end='\r', flush=True)

In [31]:
import pickle
with open('cast_info_list_total.pickle', 'wb') as to_write:
    pickle.dump(cast_info_list, to_write)

In [32]:
len(cast_info_list)

1499

In [33]:
ls

Data_Scraping_Prototype.ipynb           cleaning_prototype.ipynb
Data_Scraping_Prototype_Selenium.ipynb  geckodriver.log
Feature Engineering.ipynb               movie_info.pickle
README.md                               movie_stubs.pickle
cast_info_list1500_to_end.pickle        processed_movie_df.pickle
cast_info_list_start_to_1500.pickle     usholidays.csv
clean_movie_df.pickle


In [34]:
end = pickle.load(open('cast_info_list1500_to_end.pickle', 'rb'))

In [35]:
len(end)

992

In [48]:
len(movie_df)

2492

In [38]:
type(end)

list

In [39]:
cast_info_list.append(end)

In [50]:
cast_info

[{'title': '10,000 BC (2008)',
  'Director': 'Roland Emmerich',
  'Writer': 'Roland Emmerich, Harald Kloser',
  'Producer': 'Roland Emmerich, Mark Gordon, Michael Wimer',
  'Composer': 'Harald Kloser, Thomas Wanker',
  'Cinematographer': 'Ueli Steiger',
  'Editor': 'Alexander Berner',
  'Production Designer': 'Jean-Vincent Puzos',
  'Actors': 'Camilla Belle, Steven Strait, Marco Khan, Cliff Curtis'},
 {'title': '13 Going on 30 (2004)',
  'Director': 'Gary Winick',
  'Writer': 'Josh Goldsmith, Cathy Yuspa',
  'Producer': 'Susan Arnold, Gina Matthews, Donna Roth',
  'Composer': 'Theodore Shapiro',
  'Cinematographer': 'Don Burgess',
  'Editor': 'Susan Littenberg',
  'Production Designer': 'Garreth Stover',
  'Actors': 'Jennifer Garner, Mark Ruffalo, Judy Greer, Andy Serkis'},
 {'title': '13 Hours (2016)',
  'Director': 'Michael Bay',
  'Writer': 'Chuck Hogan, Mitchell Zuckoff',
  'Producer': 'Michael Bay, Erwin Stoff',
  'Composer': 'Lorne Balfe',
  'Cinematographer': 'Dion Beebe',
  'Ed

In [49]:
cast_info = cast_info_list + end

In [51]:
with open('cast_info.pickle', 'wb') as to_write:
    pickle.dump(cast_info, to_write)